In [ ]:
import numpy as np
import  pandas as pd



df = pd.read_csv("c:/Users/DELL/Desktop/collage/projects/solar_panel_eff_regression/dataset/train.csv")
df.head(10)


<bound method NDFrame.head of           id  temperature  irradiance            humidity  panel_age  \
0          0     7.817315  576.179270   41.24308670850264  32.135501   
1          1    24.785727  240.003973  1.3596482765960705  19.977460   
2          2    46.652695  687.612799   91.26536837560256   1.496401   
3          3    53.339567  735.141179   96.19095521176159  18.491582   
4          4     5.575374   12.241203  27.495073003585226  30.722697   
...      ...          ...         ...                 ...        ...   
19995  19995    16.868428         NaN   93.53031757838667  14.393967   
19996  19996    53.415061  296.970303   93.98571391279083  25.997012   
19997  19997     2.442727  660.328019    37.9689180401391  32.818396   
19998  19998          NaN  632.760700   43.01470184078199  19.063517   
19999  19999    25.311591  793.745224   28.91878053813607  20.073134   

       maintenance_count  soiling_ratio    voltage   current  \
0                    4.0       0.803199  

In [9]:
df.head()

,id,temperature,irradiance,humidity,panel_age,maintenance_count,soiling_ratio,voltage,current,module_temperature,cloud_coverage,wind_speed,pressure,string_id,error_code,installation_type,efficiency
0,0,7.817315,576.179270,41.24308670850264,32.135501,4.0,0.803199,37.403527,1.963787,13.691147,62.494044,12.82491203459621,1018.8665053152533,A1,NaN,NaN,0.562096
1,1,24.785727,240.003973,1.3596482765960705,19.977460,8.0,0.479456,21.843315,0.241473,27.545096,43.851238,12.012043660984917,1025.6238537572883,D4,E00,dual-axis,0.396447
2,2,46.652695,687.612799,91.26536837560256,1.496401,4.0,0.822398,48.222882,4.191800,43.363708,NaN,1.814399755560454,1010.9226539809573,C3,E00,NaN,0.573776
3,3,53.339567,735.141179,96.19095521176159,18.491582,3.0,0.837529,46.295748,0.960567,57.720436,67.361473,8.736258932034128,1021.8466633134253,A1,NaN,dual-axis,0.629009
4,4,5.575374,12.241203,27.495073003585226,30.722697,6.0,0.551833,0.000000,0.898062,6.786263,3.632000,0.52268384077164,1008.5559577591927,B2,E00,fixed,0.341874
